In [1]:
from pyecharts import options as opts
from pyecharts.globals import ThemeType
from pyecharts.charts import Line
from pyecharts.charts import Radar
from pyecharts.charts import Bar
from pyecharts.charts import Pie
from pyecharts.charts import Grid
import requests
from bs4 import BeautifulSoup
import jieba
from wordcloud import WordCloud
from matplotlib import pyplot as plt

In [ ]:
uid=eval(input())#输入up主的UID

page=eval(input())#输入up主个人空间投稿页面的某一页码

#头信息，伪装浏览器访问
header = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36'}

#返回api（url）网址中data字典里list对应值里的vlist的值
def getvlist(url):
    response = requests.get(url, headers=header)
    return response.json()["data"]["list"]["vlist"]

#返回api（url）网址中data字典里list对应值里的tlist的值
def gettlist(url):
    response = requests.get(url, headers=header)
    return response.json()["data"]["list"]["tlist"]

#获取up主的粉丝数
def getfollower(uid):
    response = requests.get("https://api.bilibili.com/x/relation/stat?vmid={}&jsonp=jsonp".format(uid), headers=header)
    return response.json()["data"]["follower"]#返回api网址中data字典里dollower对应的值

#获取up主的视频分区列表
def getregionlist(uid):
    regionList =gettlist("https://api.bilibili.com/x/space/arc/search?mid={}&ps=30&tid=0&pn=1".format(uid) + "&keyword=&order=pubdate&order_avoided=true&jsonp=jsonp")
    return regionList#一个由b站定义的代表不同分区的信息的关键字列表，如“170”对应“生活区”

#获取up主视频分区的信息
def getregionifmt(uid):
    page=1#页码保持第一页不变即可
    num=''
    r_count_dict={}
    response = requests.get("https://api.bilibili.com/x/space/arc/search?mid={}&ps=30&tid=0&pn=".format(uid) + str(page) + "&keyword=&order=pubdate&order_avoided=true&jsonp=jsonp", headers=header)
    #遍历关键字列表，根据对应字的值创建字典
    for num in getregionlist(uid):
        r_count_dict[response.json()["data"]["list"]["tlist"]["{}".format(num)]["name"]]=response.json()["data"]["list"]["tlist"]["{}".format(num)]["count"]
    return r_count_dict #返回各个投稿分区及对应投稿数的字典

#输入uid和投稿页面的某一页码，获得视频信息
def getvideoifmt(uid,page):
 videoList = getvlist("https://api.bilibili.com/x/space/arc/search?mid={}&ps=30&tid=0&pn=".format(uid) + str(
    page) + "&keyword=&order=pubdate&order_avoided=true&jsonp=jsonp")
 return videoList

#获取弹幕第一步
def get__dm(bvid):
    response = requests.get('https://api.bilibili.com/x/web-interface/view?bvid={}'.format(bvid), headers=header)#弹幕码（cid）api（url）
    return response.json()['data']['cid'] #返回代表某个视频弹幕信息的cid码，每个视频都有唯一一串cid数字

#获取弹幕第二步，输入cid
def get_dm(cid):
    url='https://comment.bilibili.com/{}.xml'.format(cid)#弹幕信息api，xml是一种传输数据的网址后缀
    response = requests.get(url,headers=header)
    soup = BeautifulSoup(response.content,'xml')#beautifulsoup解析xml网页,需导入bs4，lxml包
    dm=soup.find_all('d')#find函数获取所有d标签
    return dm #返回基础弹幕信息

#只需输入视频的bv号，结合获取第一第二步
def getdm(bvid):
    cid=get__dm(bvid)
    dm=get_dm(cid)
    return dm

#创建某一视频的弹幕词云
def getwordcloud(bvid):
    stopwords=[]
    with open("stopwords_CN.txt","r") as f:
        stopwords=f.read()#获取中文停止词
    data=[]
    counts={}
    items=[]
    #词频统计
    for item in getdm(bvid):
        data.append(item.string.split())
        data=remove_inlist(data)#去除弹幕里的嵌套列表
        #print(data)
        danmustr = ''.join(i for i in data)#讲弹幕信息转为字符串
        words = jieba.cut(danmustr)#分词
        for word in words:
            if word not in stopwords:
                counts[word]=counts.get(word,0)+1
    items=list(counts.items())#转换字典为列表

    items.sort(key=lambda x:x[1],reverse=True)#根据每一项的第二个值从大到小排序
    #print(items)
    wordstr=''
    for i in range(len(items)):
        word,count=items[i]
        wordstr+=' '+word
        #print(word,count)
    #print(wordstr)
    w=WordCloud(height = 1080,width = 1920,font_path="/Fonts/simhei.ttf").generate(wordstr)
    w.to_file(r'C:\Users\一根绿绿的葱葱\Desktop\wc.png')
    plt.imshow(w)#生成词云图

#去除列表中的嵌套列表，弹幕中有异常的嵌套列表弹幕
def remove_inlist(x):
    newlist = []
    for b in x:
        #判断列表中的项是否和list是同一类型
        if isinstance(b, list):
            for a in remove_inlist(b):
                newlist.append(a)
        else:
            newlist.append(b)
    return newlist #返回新列表


#def gettotalifmt(uid):
   # response=requests.get("https://api.bilibili.com/x/space/upstat?mid={}&jsonp=jsonp".format(uid),headers=header)
   # return response.json()["data"]["archive"]["view"],response.json()["data"]["article"]["likes"]
   #api网址显示了up主的总播放量和总点赞数，但是无法爬取，返回数据为空
    
def uppower_judge(uid):#根据up的粉丝判断up的荣誉等级
    #total_view, total_likes = gettotalifmt(uid)
    followers = getfollower(uid)#获取粉丝量
    #print("up的总播放量：",total_view)
    #print("up的总点赞数：", total_likes)
    judgement=''
    if followers < 1000:
        print("UP_POWER 鉴定为：你谁？")
        judgement = '你谁'
    elif followers >= 1000 and followers<10000:
        print("UP_POWER 鉴定为：萌新up主！")
        judgement = '萌新'
    elif followers>=10000 and followers<100000:
        print("UP_POWER 鉴定为：潜力up主！")
        judgement = '潜力'
    elif followers>=100000 and followers<1000000:
        print("UP_POWER 鉴定为：知名up主！")
        judgement = '知名'
    elif followers>=1000000:
        print("UP_POWER 鉴定为：镇站up主！")
        judgement = '镇站'
    return judgement #返回荣誉鉴定词

titles = []#视频标题列表
plays=[]#视频播放列表
video_review=[]#视频弹幕数列表
comments_count=[]#视频评论数列表
author=''#up的名字
region=[]#视频分区列表
region_count=[]#视频分区对应投稿量列表
bv_id=[]#视频的bv号
plays_video_dict={}#视频播放字典

#获取和整理视频数据
for v in getvideoifmt(uid,page):

    author=v["author"]
    video_review.append((v["video_review"]))
    comments_count.append(v["comment"])
    plays.append(v["play"])
    titles.append(v["title"])
    bv_id.append(v["bvid"])
    plays_video_dict[v["bvid"]]=v["play"]

#获取分区信息，赋值给r
r=getregionifmt(uid)

#分离分区和分区投稿量，分别存储到两个列表
for i in getregionifmt(uid):
    region.append(i)
    region_count.append(r[i])
    
#获取up投稿页面某一页的最高播放量视频的bv号
highest_play_videobid=max(plays_video_dict,key=plays_video_dict.get)

In [ ]:
bar = (
    Bar(init_opts=opts.InitOpts(theme=ThemeType.WESTEROS))#配置wesreros条形图主题
    .add_xaxis(titles)#添加x轴，值为视频标题列表
    .add_yaxis("播放量",plays,category_gap="0%")#添加y轴标题为“播放量”，值为视频播放列表，柱子图的间隔设置为0
    .add_yaxis("评论数", comments_count)#添加y轴标题为“评论数”，值为视频评论数列表
    .add_yaxis("弹幕数",video_review)#添加y轴标题为“弹幕数”，值为视频弹幕数列表
    .reversal_axis()#反转xy轴，时条形图横置
    .set_global_opts(title_opts=opts.TitleOpts(title="视频数据"))#配置条形图的标题
    .set_series_opts(markline_opts=opts.MarkLineOpts(data=[opts.MarkLineItem(type_='average',name='平均值')]))#配置条形图各数据的平均值
    .set_global_opts(datazoom_opts=opts.DataZoomOpts(type_="slider"))#设置条形图为滑动缩放
    .set_series_opts(
        label_opts=opts.LabelOpts(is_show=False),
        markpoint_opts=opts.MarkPointOpts(
            data=[
                opts.MarkPointItem(type_="min", name="最小值"),
                opts.MarkPointItem(type_="max", name="最大值"),
            ]
        )
    )#在个数据的条形图最顶部形成点标签，可代表各个数据的最大值和最小值
)#生成x轴为视频标题，y轴为播放量、评论数、弹幕数等数据的条形图

grid=Grid(init_opts=opts.InitOpts(theme=ThemeType.LIGHT))#创建格子图
grid.add(bar,grid_opts=opts.GridOpts(pos_left='40%'))#添加条形图到格子图，以调整条形图的位置，以更完整地显示视频标题
grid.render_notebook()#在notebook中渲染条形图
#gird.render()#可生成一个网址，里面是图表

In [ ]:
line = (
    Line(init_opts=opts.InitOpts(theme=ThemeType.WESTEROS))
    
    .add_xaxis(titles)
    .add_yaxis("播放量",plays)
    .add_yaxis("评论数", comments_count)
    .add_yaxis("弹幕数",video_review)
    .set_global_opts(xaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(rotate=-30)))#x轴文本倾斜，以更完整地显示视频标题
    .set_global_opts(title_opts=opts.TitleOpts(title="视频数据"))
)#生成x轴为视频标题，y轴为播放量、评论数、弹幕数等数据的折线图
line.render_notebook()
#line.render()#可生成一个网址，里面是图表

In [ ]:
datas = list(zip(region,region_count))#生成分区和分区投稿数的列表
pie=Pie()#新建一个饼图
pie.add('',datas)#添加数据
pie.set_global_opts(title_opts=opts.TitleOpts(title=" ",subtitle="视频分区"))#配置标题，由于某些up主的分区过多，造成重叠，故“视频分区”分在副标题处
pie.set_series_opts(label_opts=opts.LabelOpts(formatter=" {b}:{c} "))#配置标签
pie.render_notebook()#渲染
#pie.render()#可生成一个网址，里面是图表

In [ ]:
getwordcloud(highest_play_videobid)#获取播放量最高视频的弹幕词云

In [ ]:

w = WordCloud(height=1080, width=1920, font_path='simsun.ttc')
w.generate(' '.join(list(jieba.lcut(uppower_judge(uid)))))
w.to_file(r'C:\Users\一根绿绿的葱葱\Desktop\c.png')
plt.imshow(w)#粗糙地显示一下up荣誉鉴定